In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import os
import torch
import numpy as np
import scipy
import h5py
import matplotlib.pyplot as plt


In [ ]:
from src.utils.logger import Logging
from src.nn.tanh2 import MLP2
from src.nn.kan2 import KAN2
from src.utils.plot_losses import plot_M1_loss_history
from src.utils.simple import (
    u_momentum,
    v_momentum,
    get_rhs,
    get_coeff_mat,
    pressure_correct,
    update_velocity,
    check_divergence_free,
)

from src.models.IBMSIMPLESolver import initialize_ibm_simple_with_pinn

DEVICE = "cpu"  # torch.device("cuda" if torch.cuda.is_available() else "cpu")

# file_path = "/home/vlq26735/saiful/afrah/datasets/IBM/IB_PINN3.mat"

# h=  0.498715
# k=  0.6851750000000001
# rx=  0.209685
# ry=  0.21008499999999997

In [26]:
TEST_CHECKPOINT_PATH = "./checkpoints"
logger = Logging(TEST_CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()

logger.print(f"Device used {DEVICE}")

INFO:src.utils.logger:Device used cpu


### Load  and reshape the Testing Dataset if required

In [27]:
processed_dataset_path = "./data/Fluid_trainingData.mat"
Fluid_data = scipy.io.loadmat(processed_dataset_path)
data = h5py.File("data/IB_PINN3.mat", "r")  # load dataset from matlab
fluid = np.transpose(
    data["Fluid"], axes=range(len(data.get("Fluid").shape) - 1, -1, -1)
).astype(np.float32)

interface = Fluid_data["Solid_interface"]
solid = Fluid_data["Solid_points"]


In [28]:
tstep = 101
xstep = 102
ystep = 102

part = "Fluid"
N_data = xstep * ystep

steps = [tstep, xstep, ystep]
txy = [fluid[:, 0], fluid[:, 1], fluid[:, 2]]


input_interface = np.concatenate(
    [interface[:, 0:1], interface[:, 1:2], interface[:, 2:3]], axis=1
)


In [ ]:
model_path = "./checkpoints/2025-05-02_20-18-03-014736/model.pth"
state = torch.load(model_path)
fluid_model = MLP2(state["fluid_network"]).to("cpu")
fluid_model.load_state_dict(state["fluid_model_state_dict"])
fluid_model.eval()

# Initialize IBM SIMPLE solver with PINN
solver = initialize_ibm_simple_with_pinn(fluid_model, imax=100, jmax=100)

# Solve the FSI problem
history = solver.solve_with_ibm(max_iterations=1000, tolerance=1e-6)

# Visualize results
solver.visualize_solution()

# Plot convergence history
plt.figure(figsize=(8, 6))
plt.semilogy(history["iterations"], history["max_divergence"], "b-", linewidth=2)
plt.xlabel("Iteration")
plt.ylabel("Maximum Divergence")
plt.title("Convergence History")
plt.grid(True, alpha=0.3)
plt.show()

/home/vlq26735/saiful/afrah/code/pinn_ibm4fsi2/src/models/IBMSIMPLESolver.py:339: RuntimeWarning: invalid value encountered in sqrt
  print(f"Iteration {iteration}: Max divergence = {max_div:.2e}")
/home/vlq26735/saiful/afrah/code/pinn_ibm4fsi2/src/models/IBMSIMPLESolver.py:342: RuntimeWarning: invalid value encountered in sqrt
  print(f"Converged after {iteration + 1} iterations!")


AttributeError: 'IBMSIMPLESolver' object has no attribute 'u_momentum_with_ibm'

In [ ]:
compare_with_pinn(solver, fluid_model)


In [ ]:
visualize_advanced(solver)


In [ ]:
analyze_time_evolution(solver, fluid_model)